In [ ]:
# -*- coding: utf-8 -*-
"""
Script de Análise e Preparação de Dados para Modelagem de Machine Learning

Objetivo:
1.  Carregar e preparar os dados da carteira.
2.  Agregar os dados para o nível de CONTRATO (CCB).
3.  Criar as variáveis-alvo (flags de inadimplência para 1, 30, 60, 90 dias).
4.  Realizar uma Análise Exploratória de Dados (EDA) para entender:
    - Distribuição das variáveis-alvo (desbalanceamento).
    - Dados faltantes.
    - Cardinalidade das variáveis categóricas.
    - Relação inicial entre as features e o alvo.

Este script deve ser executado ANTES do treinamento de qualquer modelo.
"""
import pandas as pd
import numpy as np
import os

# =============================================================================
# 1. LEITURA E PREPARAÇÃO DOS DADOS (Baseado no script original)
# =============================================================================
print("="*80)
print("ETAPA 1: Carregando e limpando os dados...")
print("="*80)

#! PATHS ----------------------------------------------------------------------
#! ----------------------------------------------------------------------------
# ATENÇÃO: REDIFINIR AQUI OS PATHS PARA A SUA MÁQUINA
path_starcard = r'C:\Users\Leo\Desktop\Porto_Real\portoauto\src\originadores\data\StarCard.xlsx'
path_originadores = r'C:\Users\Leo\Desktop\Porto_Real\portoauto\src\originadores\data\Originadores.xlsx'
#------------------------------------------------------------------------------

# Carregando os dados
df_starcard = pd.read_excel(path_starcard)
cols_originadores = [
    'CCB', 'Prazo', 'Valor Parcela', 'Valor IOF', 'Valor Liquido Cliente',
    'Data Primeiro Vencimento', 'Data Último Vencimento', 'Data de Inclusão',
    'CET Mensal', 'Taxa CCB', 'Produto', 'Tabela', 'Promotora',
    'Valor Split Originador', 'Valor Split FIDC', 'Valor Split Compra de Divida',
    'Taxa Originador Split', 'Taxa Split FIDC'
]
df_originadores = pd.read_excel(path_originadores, usecols=cols_originadores)

# Unindo as fontes de dados
if not df_originadores['CCB'].is_unique:
    print("[AVISO] CCBs duplicados em Originadores.xlsx. Removendo duplicatas e mantendo a primeira ocorrência.")
    df_originadores = df_originadores.drop_duplicates(subset='CCB', keep='first')

df_merged = pd.merge(df_starcard, df_originadores, on='CCB', how='left', suffixes=('', '_orig'))

# Renomeando e limpando colunas
df_merged = df_merged.rename(columns={
    'Data Referencia': 'DataGeracao', 'Data Aquisicao': 'DataAquisicao',
    'Data Vencimento': 'DataVencimento', 'Status': 'Situacao',
    'PDD Total': 'PDDTotal', 'Valor Nominal': 'ValorNominal',
    'Valor Presente': 'ValorPresente', 'Valor Aquisicao': 'ValorAquisicao',
    'ID Cliente': 'SacadoID', 'Pagamento Parcial': 'PagamentoParcial'
})

cols_monetarias = ['ValorAquisicao', 'ValorNominal', 'ValorPresente', 'PDDTotal', 'Valor Parcela']
for col in cols_monetarias:
    if df_merged[col].dtype == 'object':
        df_merged[col] = df_merged[col].astype(str).str.replace('R$', '', regex=False).str.replace('.', '', regex=False).str.replace(',', '.', regex=False).str.strip()
        df_merged[col] = pd.to_numeric(df_merged[col], errors='coerce')

cols_data = ['DataGeracao', 'DataAquisicao', 'DataVencimento', 'Data de Nascimento']
for col in cols_data:
    df_merged[col] = pd.to_datetime(df_merged[col], errors='coerce')

print("Dados carregados e limpos.")
df_base = df_merged.copy()
del df_starcard, df_originadores, df_merged # Libera memória

# =============================================================================
# 2. ENGENHARIA DE FEATURES (Nível Parcela)
# =============================================================================
print("\n" + "="*80)
print("ETAPA 2: Engenharia de Features a nível de parcela...")
print("="*80)

# Criando dias de atraso
df_base['dias_de_atraso'] = (df_base['DataGeracao'] - df_base['DataVencimento']).dt.days

# Criando as features categóricas customizadas do script original
df_base['Produto'] = df_base['Produto'].fillna('')
df_base['Convênio'] = df_base['Convênio'].fillna('')

# _TipoProduto
condicoes_produto = [
    df_base['Produto'].str.contains('Empréstimo', case=False, na=False),
    df_base['Produto'].str.contains('Cartão RMC', case=False, na=False),
    df_base['Produto'].str.contains('Cartão Benefício', case=False, na=False)
]
opcoes_produto = ['Empréstimo', 'Cartão RMC', 'Cartão Benefício']
df_base['_TipoProduto'] = np.select(condicoes_produto, opcoes_produto, default='Outros')

# _TipoEmpregado
condicoes_empregado = [
    df_base['Produto'].str.contains('Efetivo|Efetivio', case=False, na=False, regex=True),
    df_base['Produto'].str.contains('Temporário', case=False, na=False),
    df_base['Produto'].str.contains('CONTRATADO', case=False, na=False),
    df_base['Produto'].str.contains('Comissionado', case=False, na=False)
]
opcoes_empregado = ['Efetivo', 'Temporário', 'Contratado', 'Comissionado']
df_base['_TipoEmpregado'] = np.select(condicoes_empregado, opcoes_empregado, default='Outros')

# _EsferaConvenio
condicoes_convenio = [
    df_base['Convênio'].str.contains(r'GOV\.|AGN -', case=False, na=False, regex=True),
    df_base['Convênio'].str.contains(r'PREF\.|PRERF', case=False, na=False, regex=True)
]
opcoes_convenio = ['Estadual', 'Municipal']
df_base['_EsferaConvenio'] = np.select(condicoes_convenio, opcoes_convenio, default='Outros')

# _IdadeCliente e _IdadesBins
if 'Data de Nascimento' in df_base.columns and 'DataGeracao' in df_base.columns:
    df_base['_IdadeCliente'] = ((df_base['DataGeracao'] - df_base['Data de Nascimento']).dt.days / 365.25)
    bins = [0, 37, 45, 53, 120]
    labels = ['Até 37 anos', '38 a 45 anos', '46 a 53 anos', '54 anos ou mais']
    df_base['_IdadesBins'] = pd.cut(df_base['_IdadeCliente'], bins=bins, labels=labels, right=True)

# Flags _MuitosContratos e _MuitosEntes
sacado_contratos = df_base.groupby('SacadoID')['CCB'].nunique()
sacado_contratos_alto = sacado_contratos[sacado_contratos >= 3].index
df_base['_MuitosContratos'] = df_base['SacadoID'].isin(sacado_contratos_alto).astype(int)

sacados_entes = df_base.groupby('SacadoID')['Convênio'].nunique()
sacados_entes_alto = sacados_entes[sacados_entes >= 3].index
df_base['_MuitosEntes'] = df_base['SacadoID'].isin(sacados_entes_alto).astype(int)

print("Features criadas.")

# =============================================================================
# 3. AGREGAÇÃO PARA O NÍVEL DE CONTRATO (CCB) E CRIAÇÃO DOS ALVOS
# =============================================================================
print("\n" + "="*80)
print("ETAPA 3: Agregando dados e criando variáveis-alvo...")
print("="*80)

# Agrupando por CCB para obter o máximo de dias de atraso por contrato
atraso_por_ccb = df_base.groupby('CCB')['dias_de_atraso'].max().reset_index()
atraso_por_ccb = atraso_por_ccb.rename(columns={'dias_de_atraso': 'max_dias_atraso'})

# Criando as flags de inadimplência (variáveis-alvo)
atraso_por_ccb['Vencido_1d_Flag'] = (atraso_por_ccb['max_dias_atraso'] >= 1).astype(int)
atraso_por_ccb['Vencido_30d_Flag'] = (atraso_por_ccb['max_dias_atraso'] >= 30).astype(int)
atraso_por_ccb['Vencido_60d_Flag'] = (atraso_por_ccb['max_dias_atraso'] >= 60).astype(int)
atraso_por_ccb['Vencido_90d_Flag'] = (atraso_por_ccb['max_dias_atraso'] >= 90).astype(int)

# Selecionando as features que queremos usar no modelo
# Como elas devem ser constantes por contrato, podemos pegar o primeiro valor
features_para_modelo = [
    'CCB', 'Originador', 'Convênio', 'CAPAG', 'Prazo', 'Valor Parcela',
    '_TipoProduto', '_TipoEmpregado', '_EsferaConvenio', '_IdadesBins',
    '_MuitosContratos', '_MuitosEntes'
]
# Garante que as colunas existem no dataframe antes de usar
features_existentes = [f for f in features_para_modelo if f in df_base.columns]
df_features = df_base[features_existentes].drop_duplicates(subset='CCB', keep='first')

# Unindo as features com as variáveis-alvo
df_ml = pd.merge(df_features, atraso_por_ccb, on='CCB')

# --- Engenharia de Features Adicionais (Nível Contrato) ---

# Total de Parcelas
df_ml = df_ml.rename(columns={'Prazo': 'Total_Parcelas'})

# Parcela Atual / Parcelas Restantes (Proxy)
# Contando o número de parcelas futuras para cada CCB
parcelas_futuras = df_base[df_base['dias_de_atraso'] < 0].groupby('CCB').size().reset_index(name='Parcelas_Restantes')
df_ml = pd.merge(df_ml, parcelas_futuras, on='CCB', how='left')
df_ml['Parcelas_Restantes'] = df_ml['Parcelas_Restantes'].fillna(0)
df_ml['Numero_Proxima_Parcela'] = df_ml['Total_Parcelas'] - df_ml['Parcelas_Restantes']

# TODO: Lógica para Faixa de População
# Esta feature não existe nos dados originais. Você precisaria de uma tabela
# de apoio (de-para) que mapeie Convênio ou UF para uma faixa populacional.
# Exemplo: DE-PARA: {'PREF. COTIA': '100k-500k', 'GOV. GOIAS': '>1M'}
print("[AVISO] A feature 'Faixa_Populacao' não foi criada. É necessário implementar a lógica de mapeamento.")
df_ml['Faixa_Populacao'] = 'N/A' # Placeholder

# Renomeando colunas para o padrão do pedido
df_ml = df_ml.rename(columns={
    '_EsferaConvenio': 'Esfera_Convenio',
    '_IdadesBins': 'Faixa_Idade',
    '_MuitosContratos': 'Muitos_Contratos',
    '_MuitosEntes': 'Muitos_Entes',
    '_TipoProduto': 'Tipo_Produto',
    '_TipoEmpregado': 'Tipo_Empregado',
    'Valor Parcela': 'Valor_Parcela',
})

print(f"Dataset para ML criado com {df_ml.shape[0]} contratos (linhas) e {df_ml.shape[1]} features (colunas).")


# =============================================================================
# 4. ANÁLISE EXPLORATÓRIA DE DADOS (EDA) - "DEBUGGING"
# =============================================================================
print("\n" + "="*80)
print("ETAPA 4: Análise Exploratória dos Dados para ML")
print("="*80)

# --- 4.1 Informações Gerais e Dados Faltantes ---
print("\n--- 4.1 Informações Gerais e Dados Faltantes ---")
print("df_ml.info():")
df_ml.info()

missing_values = df_ml.isnull().sum()
missing_percent = (missing_values / len(df_ml) * 100).round(2)
missing_df = pd.DataFrame({'Valores Faltantes': missing_values, '% Faltante': missing_percent})
print("\nAnálise de Dados Faltantes:")
print(missing_df[missing_df['Valores Faltantes'] > 0])

# --- 4.2 Análise do Desbalanceamento das Classes ---
print("\n--- 4.2 Análise do Desbalanceamento das Classes (Variáveis-Alvo) ---")
targets = ['Vencido_1d_Flag', 'Vencido_30d_Flag', 'Vencido_60d_Flag', 'Vencido_90d_Flag']
for target in targets:
    print(f"\nDistribuição para '{target}':")
    counts = df_ml[target].value_counts()
    percentages = df_ml[target].value_counts(normalize=True) * 100
    dist_df = pd.DataFrame({'Contagem': counts, 'Percentual (%)': percentages.round(2)})
    print(dist_df)
    print("-"*30)

# --- 4.3 Análise de Cardinalidade das Variáveis Categóricas ---
print("\n--- 4.3 Análise de Cardinalidade (Valores Únicos) das Features Categóricas ---")
categorical_features = df_ml.select_dtypes(include=['object', 'category']).columns
for feature in categorical_features:
    unique_count = df_ml[feature].nunique()
    print(f"- '{feature}': {unique_count} valores únicos.")
    if unique_count > 50:
        print(f"  [ATENÇÃO] Alta cardinalidade! Pode ser necessário agrupar ou usar Target Encoding.")

# --- 4.4 Análise Bivariada (Relação Feature vs. Alvo) ---
print("\n--- 4.4 Análise Bivariada: Taxa de Inadimplência por Categoria ---")
print("Analisando a taxa de inadimplência para o alvo 'Vencido_30d_Flag'")

features_to_analyze = [
    'Esfera_Convenio', 'Faixa_Populacao', 'CAPAG', 'Muitos_Contratos',
    'Tipo_Produto', 'Tipo_Empregado', 'Faixa_Idade', 'Originador', 'Muitos_Entes'
]
features_to_analyze = [f for f in features_to_analyze if f in df_ml.columns] # Garantir que existem

for feature in features_to_analyze:
    # Ignora features com muitos valores únicos para não poluir a saída
    if df_ml[feature].nunique() > 30 and df_ml[feature].dtype == 'object':
        print(f"\nFeature '{feature}' tem alta cardinalidade ({df_ml[feature].nunique()}) - pulando a exibição detalhada.")
        continue

    print(f"\n--- Taxa de inadimplência (Vencido_30d_Flag) por '{feature}' ---")
    # Calcula a média da flag (que é a taxa de inadimplência) e a contagem de contratos
    analysis = df_ml.groupby(feature)['Vencido_30d_Flag'].agg(['mean', 'count']).reset_index()
    analysis['mean'] = (analysis['mean'] * 100).round(2)
    analysis = analysis.rename(columns={'mean': 'Taxa Inadimplência (%)', 'count': 'Nº Contratos'})
    print(analysis.sort_values(by='Taxa Inadimplência (%)', ascending=False))


print("\n" + "="*80)
print("ANÁLISE DE PREPARAÇÃO CONCLUÍDA!")
print("Use os insights acima para definir a estratégia de pré-processamento e modelagem.")
print("="*80)

In [1]:
%pip install pip install pandas numpy scikit-learn xgboost lightgbm

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement install (from versions: none)
ERROR: No matching distribution found for install
You should consider upgrading via the 'c:\Users\Leo\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [5]:
%pip install lightgbm


You should consider upgrading via the 'c:\Users\Leo\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [26]:
# -*- coding: utf-8 -*-
"""
Script de Treinamento e Avaliação de Modelos de Machine Learning
VERSÃO COM MELHORIAS IMPLEMENTADAS
"""
import pandas as pd
import numpy as np
import os
import warnings

# Modelos
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb
import lightgbm as lgb

# Métricas
from sklearn.metrics import roc_auc_score

warnings.filterwarnings('ignore')

# =============================================================================
# ETAPA 1: CARREGAMENTO E PREPARAÇÃO DOS DADOS (Função consolidada)
# =============================================================================
def carregar_e_preparar_dados_corrigido():
    """
    Função corrigida para não criar as features que causam vazamento de dados
    e para calcular a feature _MuitosContratos de forma temporalmente consciente.
    """
    print("="*80)
    print("ETAPA 1: Carregando e preparando os dados (VERSÃO CORRIGIDA)...")
    print("="*80)

    #! PATHS ----------------------------------------------------------------------
    # ATENÇÃO: REDIFINIR AQUI OS PATHS PARA A SUA MÁQUINA
    path_starcard = r'C:\Users\Leo\Desktop\Porto_Real\portoauto\src\originadores\data\StarCard.xlsx'
    path_originadores = r'C:\Users\Leo\Desktop\Porto_Real\portoauto\src\originadores\data\Originadores.xlsx'
    #------------------------------------------------------------------------------

    df_starcard = pd.read_excel(path_starcard)
    
    # <--- ALTERAÇÃO 1: Incluir 'Data de Inclusão' na lista de colunas a serem carregadas
    cols_originadores = ['CCB', 'Data de Inclusão', 'Prazo', 'Valor Parcela', 'Produto', 'Convênio']
    df_originadores = pd.read_excel(path_originadores, usecols=cols_originadores)

    if not df_originadores['CCB'].is_unique:
        df_originadores = df_originadores.drop_duplicates(subset='CCB', keep='first')

    df_merged = pd.merge(df_starcard, df_originadores, on='CCB', how='left', suffixes=('', '_orig'))

    df_merged = df_merged.rename(columns={
        'Data Referencia': 'DataGeracao', 'Data Vencimento': 'DataVencimento',
        'ID Cliente': 'SacadoID'
    })

    cols_monetarias = ['Valor Parcela']
    for col in cols_monetarias:
        if df_merged[col].dtype == 'object':
            df_merged[col] = df_merged[col].astype(str).str.replace('R$', '', regex=False).str.replace('.', '', regex=False).str.replace(',', '.', regex=False).str.strip()
            df_merged[col] = pd.to_numeric(df_merged[col], errors='coerce')

    # <--- ALTERAÇÃO 2: Garantir que 'Data de Inclusão' seja tratada como data
    cols_data = ['DataGeracao', 'DataVencimento', 'Data de Nascimento', 'Data de Inclusão']
    for col in cols_data:
        # Adicionado um verificador para não dar erro se a coluna não existir
        if col in df_merged.columns:
            df_merged[col] = pd.to_datetime(df_merged[col], errors='coerce')

    df_base = df_merged.copy()
    del df_starcard, df_originadores, df_merged

    df_base['dias_de_atraso'] = (df_base['DataGeracao'] - df_base['DataVencimento']).dt.days
    df_base['Produto'] = df_base['Produto'].fillna('')
    df_base['Convênio'] = df_base['Convênio'].fillna('')
    condicoes_produto = [df_base['Produto'].str.contains('Empréstimo', case=False, na=False), df_base['Produto'].str.contains('Cartão RMC', case=False, na=False), df_base['Produto'].str.contains('Cartão Benefício', case=False, na=False)]
    opcoes_produto = ['Empréstimo', 'Cartão RMC', 'Cartão Benefício']
    df_base['_TipoProduto'] = np.select(condicoes_produto, opcoes_produto, default='Outros')
    condicoes_empregado = [df_base['Produto'].str.contains('Efetivo|Efetivio', case=False, na=False, regex=True), df_base['Produto'].str.contains('Temporário', case=False, na=False), df_base['Produto'].str.contains('CONTRATADO', case=False, na=False), df_base['Produto'].str.contains('Comissionado', case=False, na=False)]
    opcoes_empregado = ['Efetivo', 'Temporário', 'Contratado', 'Comissionado']
    df_base['_TipoEmpregado'] = np.select(condicoes_empregado, opcoes_empregado, default='Outros')
    condicoes_convenio = [df_base['Convênio'].str.contains(r'GOV\.|AGN -', case=False, na=False, regex=True), df_base['Convênio'].str.contains(r'PREF\.|PRERF', case=False, na=False, regex=True)]
    opcoes_convenio = ['Estadual', 'Municipal']
    df_base['_EsferaConvenio'] = np.select(condicoes_convenio, opcoes_convenio, default='Outros')
    if 'Data de Nascimento' in df_base.columns:
        df_base['_IdadeCliente'] = ((df_base['DataGeracao'] - df_base['Data de Nascimento']).dt.days / 365.25)
        bins = [0, 37, 45, 53, 120]
        labels = ['Até 37 anos', '38 a 45 anos', '46 a 53 anos', '54 anos ou mais']
        df_base['_IdadesBins'] = pd.cut(df_base['_IdadeCliente'], bins=bins, labels=labels, right=True)

    # Lógica temporalmente consciente para '_MuitosContratos'
    df_base = df_base.sort_values(by=['SacadoID', 'Data de Inclusão'])
    df_base['_NumContratosAnteriores'] = df_base.groupby('SacadoID').cumcount()
    df_base['_MuitosContratos'] = (df_base['_NumContratosAnteriores'] >= 2).astype(int)

    atraso_por_ccb = df_base.groupby('CCB')['dias_de_atraso'].max().reset_index()
    atraso_por_ccb = atraso_por_ccb.rename(columns={'dias_de_atraso': 'max_dias_atraso'})
    atraso_por_ccb['Vencido_1d_Flag'] = (atraso_por_ccb['max_dias_atraso'] >= 1).astype(int)
    atraso_por_ccb['Vencido_30d_Flag'] = (atraso_por_ccb['max_dias_atraso'] >= 30).astype(int)
    atraso_por_ccb['Vencido_60d_Flag'] = (atraso_por_ccb['max_dias_atraso'] >= 60).astype(int)
    atraso_por_ccb['Vencido_90d_Flag'] = (atraso_por_ccb['max_dias_atraso'] >= 90).astype(int)

    features_para_modelo = ['CCB', 'Convênio', 'CAPAG', 'Prazo', 'Valor Parcela', '_TipoProduto', '_TipoEmpregado', '_EsferaConvenio', '_IdadesBins', '_MuitosContratos']
    features_existentes = [f for f in features_para_modelo if f in df_base.columns]
    
    df_features = df_base[features_existentes].drop_duplicates(subset='CCB', keep='first')
    df_ml = pd.merge(df_features, atraso_por_ccb, on='CCB')
    
    df_ml = df_ml.rename(columns={
        'Prazo': 'Total_Parcelas', 'Valor Parcela': 'Valor_Parcela', '_EsferaConvenio': 'Esfera_Convenio',
        '_IdadesBins': 'Faixa_Idade', '_MuitosContratos': 'Muitos_Contratos',
        '_TipoProduto': 'Tipo_Produto', '_TipoEmpregado': 'Tipo_Empregado'
    })
    print("Dataset para ML criado (com feature de múltiplos contratos temporalmente correta).")
    return df_ml

# =============================================================================
# ETAPA 2: TREINAMENTO E AVALIAÇÃO (COM MELHORIAS)
# =============================================================================

#df_ml = carregar_e_preparar_dados_corrigido()

targets = ['Vencido_1d_Flag', 'Vencido_30d_Flag', 'Vencido_60d_Flag', 'Vencido_90d_Flag']
resultados_finais = pd.DataFrame()

for target in targets:
    print("\n" + "="*80)
    print(f"   TREINANDO MODELOS PARA O ALVO: {target} (COM MELHORIAS)")
    print("="*80)

    X = df_ml.drop(columns=targets + ['CCB', 'max_dias_atraso'])
    y = df_ml[target]

    numerical_features = X.select_dtypes(include=np.number).columns.tolist()
    categorical_features = X.select_dtypes(include=['object', 'category']).columns.tolist()

    # A Dummification foi movida para DEPOIS do train_test_split
    # X = pd.get_dummies(X, columns=categorical_features, drop_first=True)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42, stratify=y
    )
    
    # --- MELHORIA 2: Aplicar Dummification APÓS a divisão Treino/Teste ---
    # Isso garante que nenhuma informação sobre categorias do conjunto de teste vaze para o treino.
    X_train = pd.get_dummies(X_train, columns=categorical_features, drop_first=True)
    X_test = pd.get_dummies(X_test, columns=categorical_features, drop_first=True)

    # Alinha as colunas para garantir que treino e teste tenham as mesmas features.
    # Isso lida com casos onde uma categoria pode aparecer no treino mas não no teste ou vice-versa.
    training_columns = X_train.columns
    X_test = X_test.reindex(columns=training_columns, fill_value=0)
    
    # É importante garantir que as colunas numéricas para o scaler sejam as que existem APÓS o dummification
    numerical_features_after_dummies = [col for col in numerical_features if col in X_train.columns]

    scaler = StandardScaler()
    X_train[numerical_features_after_dummies] = scaler.fit_transform(X_train[numerical_features_after_dummies])
    X_test[numerical_features_after_dummies] = scaler.transform(X_test[numerical_features_after_dummies])

    print(f"Dados divididos: {len(X_train)} para treino, {len(X_test)} para teste.")

    scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()

    models = {
        "Regressão Logística": LogisticRegression(random_state=42, class_weight='balanced', max_iter=1000),
        "Random Forest": RandomForestClassifier(random_state=42, class_weight='balanced', n_jobs=-1),
        # --- MELHORIA 3: Alterado para kernel 'rbf' para capturar relações não-lineares ---
        "SVM (Kernel RBF)": SVC(kernel='rbf', probability=True, random_state=42, class_weight='balanced'),
        "XGBoost": xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss', scale_pos_weight=scale_pos_weight, n_jobs=-1),
        "LightGBM": lgb.LGBMClassifier(random_state=42, scale_pos_weight=scale_pos_weight, n_jobs=-1)
    }

    resultados_target = {}
    for name, model in models.items():
        print(f"\n--- Treinando {name} ---")
        model.fit(X_train, y_train)
        y_pred_proba = model.predict_proba(X_test)[:, 1]
        auc = roc_auc_score(y_test, y_pred_proba)
        resultados_target[name] = auc
        print(f"ROC-AUC Score: {auc:.4f}")

        if hasattr(model, 'feature_importances_'):
            importances = pd.DataFrame({
                'Feature': X_train.columns,
                'Importance': model.feature_importances_
            }).sort_values('Importance', ascending=False).head(10)
            print("Top 10 Features mais importantes:")
            print(importances)

    df_resultados_target = pd.DataFrame.from_dict(
        resultados_target, orient='index', columns=[target]
    )
    resultados_finais = pd.concat([resultados_finais, df_resultados_target], axis=1)

# =============================================================================
# ETAPA 3: EXIBIÇÃO DO RESUMO FINAL
# =============================================================================
print("\n" + "="*80)
print("   RESUMO COMPARATIVO FINAL - ROC-AUC SCORE (COM MELHORIAS)")
print("="*80)
print(resultados_finais.round(4))

melhor_modelo_geral = resultados_finais.mean(axis=1).idxmax()
print("\n" + "-"*80)
print(f"Considerando a média de desempenho em todos os alvos, o melhor modelo parece ser: '{melhor_modelo_geral}'")
print("-"*80)


   TREINANDO MODELOS PARA O ALVO: Vencido_1d_Flag (COM MELHORIAS)
Dados divididos: 4786 para treino, 2052 para teste.

--- Treinando Regressão Logística ---
ROC-AUC Score: 0.9321

--- Treinando Random Forest ---
ROC-AUC Score: 0.9297
Top 10 Features mais importantes:
                      Feature  Importance
0              Total_Parcelas    0.208804
1               Valor_Parcela    0.174818
34  Tipo_Empregado_Contratado    0.151244
37  Tipo_Empregado_Temporário    0.112349
35     Tipo_Empregado_Efetivo    0.051624
33    Tipo_Produto_Empréstimo    0.030710
3         Convênio_GOV. GOIAS    0.025748
38  Esfera_Convenio_Municipal    0.024749
2            Muitos_Contratos    0.020859
32    Tipo_Produto_Cartão RMC    0.019955

--- Treinando SVM (Kernel RBF) ---
ROC-AUC Score: 0.9217

--- Treinando XGBoost ---
ROC-AUC Score: 0.9419
Top 10 Features mais importantes:
                             Feature  Importance
34         Tipo_Empregado_Contratado    0.368344
38         Esfera_Convenio_Mun

In [ ]:
# -*- coding: utf-8 -*-
"""
Script de Treinamento e Avaliação de Modelos de Machine Learning
VERSÃO FINAL — ancorado em 'Data de Inclusão', sem vazamentos, com checagens
"""
import pandas as pd
import numpy as np
import os
import warnings

# Modelos
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb
import lightgbm as lgb

# Métricas
from sklearn.metrics import roc_auc_score, average_precision_score

# Pré-processamento
from sklearn.impute import SimpleImputer

warnings.filterwarnings('ignore')


# =============================================================================
# ETAPA 1: CARREGAMENTO E PREPARAÇÃO DOS DADOS
# =============================================================================
def carregar_e_preparar_dados_corrigido():
    """
    1) Lê StarCard.xlsx e Originadores.xlsx
    2) Usa 'Data de Inclusão' como relógio:
       - snapshot determinístico por CCB = menor Data de Inclusão disponível
       - _MuitosContratos por ordenação temporal (cumcount)
       - Idade calculada na Data de Inclusão (fallback em DataGeracao se nula)
    3) Cria labels 'Vencido_{1,30,60,90}d' com base em dias_de_atraso no snapshot.
    4) Evita vazamentos: não inclui colunas temporais/labels em X.
    """
    print("="*88)
    print("ETAPA 1: Carregando e preparando os dados (VERSÃO FINAL)...")
    print("="*88)

    #! PATHS ----------------------------------------------------------------------
    # ATENÇÃO: REDIFINIR AQUI OS PATHS PARA A SUA MÁQUINA
    path_starcard = r'C:\Users\Leo\Desktop\Porto_Real\portoauto\src\originadores\data\StarCard.xlsx'
    path_originadores = r'C:\Users\Leo\Desktop\Porto_Real\portoauto\src\originadores\data\Originadores.xlsx'
    #------------------------------------------------------------------------------

    # ---------- Leitura ----------
    df_starcard = pd.read_excel(path_starcard)
    cols_originadores = ['CCB', 'Data de Inclusão', 'Prazo', 'Valor Parcela', 'Produto', 'Convênio', 'CAPAG']
    # CAPAG pode não existir; vamos ler com try simples
    try:
        df_originadores = pd.read_excel(path_originadores, usecols=cols_originadores)
    except ValueError:
        # remove CAPAG se não existir
        cols_originadores = [c for c in cols_originadores if c != 'CAPAG']
        df_originadores = pd.read_excel(path_originadores, usecols=cols_originadores)

    # Normaliza datas e unicidade por CCB (primeira inclusão)
    if 'Data de Inclusão' in df_originadores.columns:
        df_originadores['Data de Inclusão'] = pd.to_datetime(df_originadores['Data de Inclusão'], errors='coerce')
        df_originadores = (
            df_originadores
            .sort_values(['CCB', 'Data de Inclusão'])
            .drop_duplicates(subset='CCB', keep='first')
        )

    # Merge
    df_merged = pd.merge(df_starcard, df_originadores, on='CCB', how='left', suffixes=('', '_orig'))

    # Renomeios
    df_merged = df_merged.rename(columns={
        'Data Referencia': 'DataGeracao',
        'Data Vencimento': 'DataVencimento',
        'ID Cliente': 'SacadoID'
    })

    # Tipos de data
    for col in ['DataGeracao', 'DataVencimento', 'Data de Nascimento']:
        if col in df_merged.columns:
            df_merged[col] = pd.to_datetime(df_merged[col], errors='coerce')

    # Conversão monetária
    if 'Valor Parcela' in df_merged.columns and df_merged['Valor Parcela'].dtype == 'object':
        s = df_merged['Valor Parcela'].astype(str)
        s = s.str.replace('R$', '', regex=False).str.replace('.', '', regex=False).str.replace(',', '.', regex=False).str.strip()
        df_merged['Valor Parcela'] = pd.to_numeric(s, errors='coerce')

    # Base
    df_base = df_merged.copy()
    del df_starcard, df_originadores, df_merged

    # Limpezas básicas
    if 'Produto' in df_base.columns:
        df_base['Produto'] = df_base['Produto'].fillna('')
    if 'Convênio' in df_base.columns:
        df_base['Convênio'] = df_base['Convênio'].fillna('')

    # Feature dias_de_atraso (com DataGeracao do snapshot; se DataGeracao é constante, vira “situação no dia da extração”)
    if 'DataGeracao' in df_base.columns and 'DataVencimento' in df_base.columns:
        df_base['dias_de_atraso'] = (df_base['DataGeracao'] - df_base['DataVencimento']).dt.days

    # Buckets de produto/empregado/convênio
    condicoes_produto = [
        df_base['Produto'].str.contains('Empréstimo', case=False, na=False),
        df_base['Produto'].str.contains('Cartão RMC', case=False, na=False),
        df_base['Produto'].str.contains('Cartão Benefício', case=False, na=False)
    ]
    opcoes_produto = ['Empréstimo', 'Cartão RMC', 'Cartão Benefício']
    df_base['_TipoProduto'] = np.select(condicoes_produto, opcoes_produto, default='Outros')

    condicoes_empregado = [
        df_base['Produto'].str.contains('Efetivo|Efetivio', case=False, na=False, regex=True),
        df_base['Produto'].str.contains('Temporário', case=False, na=False),
        df_base['Produto'].str.contains('CONTRATADO', case=False, na=False),
        df_base['Produto'].str.contains('Comissionado', case=False, na=False)
    ]
    opcoes_empregado = ['Efetivo', 'Temporário', 'Contratado', 'Comissionado']
    df_base['_TipoEmpregado'] = np.select(condicoes_empregado, opcoes_empregado, default='Outros')

    # Regex de convênio (ajustada para 'PREF' genérico; mantém 'AGN -' e 'GOV.')
    condicoes_convenio = [
        df_base['Convênio'].str.contains(r'GOV\.|AGN -', case=False, na=False, regex=True),
        df_base['Convênio'].str.contains(r'PREF', case=False, na=False, regex=True)
    ]
    opcoes_convenio = ['Estadual', 'Municipal']
    df_base['_EsferaConvenio'] = np.select(condicoes_convenio, opcoes_convenio, default='Outros')

    # ----- Relógio: Data de Inclusão -----
    if 'Data de Inclusão' not in df_base.columns:
        raise ValueError("Coluna 'Data de Inclusão' não encontrada após o merge. Verifique Originadores.xlsx.")

    # Ordenação temporal por inclusão para _MuitosContratos
    df_base = df_base.sort_values(['SacadoID', 'Data de Inclusão', 'CCB'])
    df_base['_NumContratosAnteriores'] = df_base.groupby('SacadoID').cumcount()
    df_base['_MuitosContratos'] = (df_base['_NumContratosAnteriores'] >= 2).astype(int)

    # Idade na Data de Inclusão (fallback em DataGeracao, se inclusão nula)
    if 'Data de Nascimento' in df_base.columns:
        base_idade = df_base['Data de Inclusão'].fillna(df_base.get('DataGeracao'))
        df_base['_IdadeCliente'] = (base_idade - df_base['Data de Nascimento']).dt.days / 365.25
        bins = [0, 37, 45, 53, 120]
        labels = ['Até 37 anos', '38 a 45 anos', '46 a 53 anos', '54 anos ou mais']
        df_base['_IdadesBins'] = pd.cut(df_base['_IdadeCliente'], bins=bins, labels=labels, right=True)

    # ---------- Labels por CCB ----------
    # Se tiver múltiplas linhas por CCB (parcelas/linhas), agregue o maior atraso
    if 'dias_de_atraso' not in df_base.columns:
        raise ValueError("'dias_de_atraso' não foi criado. Verifique 'DataGeracao' e 'DataVencimento'.")

    atraso_por_ccb = (
        df_base.groupby('CCB', as_index=False)['dias_de_atraso'].max()
        .rename(columns={'dias_de_atraso': 'max_dias_atraso'})
    )
    for k in [1, 30, 60, 90]:
        atraso_por_ccb[f'Vencido_{k}d_Flag'] = (atraso_por_ccb['max_dias_atraso'] >= k).astype(int)

    # ---------- Snapshot determinístico por CCB ----------
    # referência temporal: Data de Inclusão (fallback DataGeracao)
    df_base['DataInclRef'] = df_base['Data de Inclusão'].fillna(df_base.get('DataGeracao'))
    snapshots = df_base.sort_values(['CCB', 'DataInclRef', 'DataVencimento'])
    primeiro_snap = snapshots.groupby('CCB', as_index=False).first()

    features_para_modelo = [
        'CCB', 'Convênio', 'CAPAG', 'Prazo', 'Valor Parcela',
        '_TipoProduto', '_TipoEmpregado', '_EsferaConvenio', '_IdadesBins', '_MuitosContratos'
    ]
    ausentes = sorted(list(set(features_para_modelo) - set(primeiro_snap.columns)))
    if ausentes:
        print(f"[AVISO] Features ausentes e removidas (não encontradas no snapshot): {ausentes}")

    features_existentes = [f for f in features_para_modelo if f in primeiro_snap.columns]
    df_features = primeiro_snap[features_existentes]

    # df_ml final
    df_ml = pd.merge(df_features, atraso_por_ccb, on='CCB', how='inner').rename(columns={
        'Prazo': 'Total_Parcelas',
        'Valor Parcela': 'Valor_Parcela',
        '_EsferaConvenio': 'Esfera_Convenio',
        '_IdadesBins': 'Faixa_Idade',
        '_MuitosContratos': 'Muitos_Contratos',
        '_TipoProduto': 'Tipo_Produto',
        '_TipoEmpregado': 'Tipo_Empregado'
    })

    # Checagens úteis
    assert df_ml['CCB'].is_unique, "CCB não está único no df_ml! Revise o snapshot por CCB."
    assert df_ml[['Vencido_1d_Flag','Vencido_30d_Flag','Vencido_60d_Flag','Vencido_90d_Flag']].notna().all().all(), \
        "Algum label Vencido_* está com NaN."

    print("Dataset para ML criado com 1 linha por CCB (snapshot na Data de Inclusão) e labels agregados.")
    return df_ml


# =============================================================================
# ETAPA 2: TREINAMENTO E AVALIAÇÃO
# =============================================================================
def treinar_e_avaliar(df_ml):
    targets = ['Vencido_1d_Flag', 'Vencido_30d_Flag', 'Vencido_60d_Flag', 'Vencido_90d_Flag']
    resultados_finais = pd.DataFrame()

    for target in targets:
        print("\n" + "="*88)
        print(f"   TREINANDO MODELOS PARA O ALVO: {target}")
        print("="*88)

        # Define X e y removendo colunas proibidas
        X = df_ml.drop(columns=targets + ['CCB', 'max_dias_atraso'])
        y = df_ml[target]

        # Identifica colunas numéricas originais (antes de dummies)
        numerical_features = X.select_dtypes(include=np.number).columns.tolist()
        categorical_features = X.select_dtypes(include=['object', 'category']).columns.tolist()

        # Split estratificado
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.30, random_state=42, stratify=y
        )

        # Dummification APÓS o split + alinhamento
        X_train = pd.get_dummies(X_train, columns=categorical_features, drop_first=True)
        X_test  = pd.get_dummies(X_test,  columns=categorical_features, drop_first=True)
        X_test  = X_test.reindex(columns=X_train.columns, fill_value=0)

        # Guardas básicas
        print("#cols treino:", X_train.shape[1], "| #cols teste:", X_test.shape[1])
        assert list(X_train.columns) == list(X_test.columns), "Treino e teste desalinhados!"

        # Imputação + Escalonamento SOMENTE nas numéricas originais (interseção)
        num_cols = [c for c in numerical_features if c in X_train.columns]
        if len(num_cols) > 0:
            imputer = SimpleImputer(strategy='median')
            X_train[num_cols] = imputer.fit_transform(X_train[num_cols])
            X_test[num_cols]  = imputer.transform(X_test[num_cols])

            scaler = StandardScaler()
            X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
            X_test[num_cols]  = scaler.transform(X_test[num_cols])

        # Log base rates
        print("Base rate no treino:", float(y_train.mean()).__round__(4), "| no teste:", float(y_test.mean()).__round__(4))

        # scale_pos_weight com guarda
        pos = int((y_train == 1).sum())
        neg = int((y_train == 0).sum())
        if pos == 0:
            print(f"[AVISO] Classe positiva ausente no treino para {target}. Ajustando scale_pos_weight=1.0.")
            scale_pos_weight = 1.0
        else:
            scale_pos_weight = neg / pos

        # Modelos
        models = {
            "Regressão Logística": LogisticRegression(random_state=42, class_weight='balanced', max_iter=1000),
            "Random Forest":       RandomForestClassifier(random_state=42, class_weight='balanced', n_jobs=-1),
            "SVM (Kernel RBF)":    SVC(kernel='rbf', probability=True, random_state=42, class_weight='balanced'),
            "XGBoost":             xgb.XGBClassifier(random_state=42, eval_metric='logloss',
                                                     use_label_encoder=False, scale_pos_weight=scale_pos_weight,
                                                     n_jobs=-1),
            "LightGBM":            lgb.LGBMClassifier(random_state=42, scale_pos_weight=scale_pos_weight,
                                                      n_jobs=-1)
        }

        resultados_target = {}
        for name, model in models.items():
            print(f"\n--- Treinando {name} ---")
            model.fit(X_train, y_train)
            # Probabilidades
            y_pred_proba = model.predict_proba(X_test)[:, 1]
            # Métricas
            auc = roc_auc_score(y_test, y_pred_proba)
            ap  = average_precision_score(y_test, y_pred_proba)
            resultados_target[name] = {'ROC-AUC': auc, 'PR-AUC(AP)': ap}
            print(f"ROC-AUC: {auc:.4f} | PR-AUC(AP): {ap:.4f}")

            # Importâncias — árvores/GBMs
            if isinstance(model, xgb.XGBClassifier):
                booster = model.get_booster()
                imp = booster.get_score(importance_type='gain')
                if len(imp) > 0:
                    importances = (pd.Series(imp).sort_values(ascending=False).head(10)
                                   .rename('Gain').reset_index().rename(columns={'index': 'Feature'}))
                    print("Top 10 importâncias (XGB, gain):")
                    print(importances)
            elif isinstance(model, lgb.LGBMClassifier):
                feats = model.feature_name_
                gains = model.booster_.feature_importance(importance_type='gain')
                importances = (pd.DataFrame({'Feature': feats, 'Gain': gains})
                               .sort_values('Gain', ascending=False).head(10))
                print("Top 10 importâncias (LGBM, gain):")
                print(importances)
            elif hasattr(model, 'feature_importances_'):
                importances = pd.DataFrame({
                    'Feature': X_train.columns,
                    'Importance': model.feature_importances_
                }).sort_values('Importance', ascending=False).head(10)
                print("Top 10 importâncias:")
                print(importances)
            elif isinstance(model, LogisticRegression):
                coefs = pd.Series(model.coef_[0], index=X_train.columns).sort_values(key=np.abs, ascending=False).head(10)
                print("Top 10 |coef| (Regressão Logística):")
                print(coefs)

        # Acumula resultados por alvo
        df_resultados_target = (pd.DataFrame(resultados_target).T)
        # guarda apenas ROC-AUC numa tabela comparativa “clássica”
        resultados_finais = pd.concat([resultados_finais, df_resultados_target['ROC-AUC'].rename(target)], axis=1)

        # Checagens adicionais úteis
        suspeitas = {'dias_de_atraso', 'max_dias_atraso', 'DataGeracao', 'DataVencimento', 'Data de Inclusão'}
        suspeitas_presentes = sorted(list(set(X_train.columns) & suspeitas))
        if suspeitas_presentes:
            print(f"[ALERTA] Colunas suspeitas presentes em X: {suspeitas_presentes}")

    return resultados_finais


# =============================================================================
# ETAPA 3: EXECUÇÃO
# =============================================================================
if __name__ == "__main__":
    df_ml = carregar_e_preparar_dados_corrigido()

    # Relatório rápido de base rates por label
    print("\nBase rates dos alvos (df_ml):")
    print(df_ml[['Vencido_1d_Flag','Vencido_30d_Flag','Vencido_60d_Flag','Vencido_90d_Flag']].mean().round(4))

    resultados_finais = treinar_e_avaliar(df_ml)

    print("\n" + "="*88)
    print("   RESUMO COMPARATIVO FINAL - ROC-AUC")
    print("="*88)
    print(resultados_finais.round(4))

    melhor_modelo_geral = resultados_finais.mean(axis=1).idxmax()
    print("\n" + "-"*88)
    print(f"Considerando a média de ROC-AUC em todos os alvos, o melhor modelo parece ser: '{melhor_modelo_geral}'")
    print("-"*88)


## esse aqui estava bom, mas nao reconhece o CAPAG e nao depura direito

ETAPA 1: Carregando e preparando os dados (VERSÃO FINAL)...
Dataset para ML criado com 1 linha por CCB (snapshot na Data de Inclusão) e labels agregados.

Base rates dos alvos (df_ml):
Vencido_1d_Flag     0.8157
Vencido_30d_Flag    0.6897
Vencido_60d_Flag    0.5294
Vencido_90d_Flag    0.3353
dtype: float64

   TREINANDO MODELOS PARA O ALVO: Vencido_1d_Flag
#cols treino: 43 | #cols teste: 43
Base rate no treino: 0.8157 | no teste: 0.8158

--- Treinando Regressão Logística ---
ROC-AUC: 0.9372 | PR-AUC(AP): 0.9819
Top 10 |coef| (Regressão Logística):
Tipo_Empregado_Contratado         -4.054948
Convênio_PREF. ANANINDEUA         -3.682524
Tipo_Empregado_Temporário          3.488191
Tipo_Empregado_Efetivo             2.488744
Convênio_PREF. CAMPOS DO JORDÃO    2.305191
Tipo_Produto_Empréstimo           -1.713887
Convênio_PREF. COTIA               1.706626
Convênio_PREF. EMBU DAS ARTES     -1.546657
Convênio_PREF. SANTA LUZIA         1.468440
Esfera_Convenio_Municipal         -1.463444
dtype:

In [29]:
# -*- coding: utf-8 -*-
"""
Script de Treinamento e Avaliação de Modelos de Machine Learning
VERSÃO FINAL — ancorado em 'Data de Inclusão', sem vazamentos, com checagens
"""
import pandas as pd
import numpy as np
import os
import warnings

# Modelos
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb
import lightgbm as lgb

# Métricas
from sklearn.metrics import roc_auc_score, average_precision_score

# Pré-processamento
from sklearn.impute import SimpleImputer

warnings.filterwarnings('ignore')


# =============================================================================
# ETAPA 1: CARREGAMENTO E PREPARAÇÃO DOS DADOS
# =============================================================================
def carregar_e_preparar_dados_corrigido():
    """
    1) Lê StarCard.xlsx e Originadores.xlsx
    2) Usa 'Data de Inclusão' como relógio:
        - snapshot determinístico por CCB = menor Data de Inclusão disponível
        - _MuitosContratos por ordenação temporal (cumcount)
        - Idade calculada na Data de Inclusão (fallback em DataGeracao se nula)
    3) Cria labels 'Vencido_{1,30,60,90}d' com base em dias_de_atraso no snapshot.
    4) Evita vazamentos: não inclui colunas temporais/labels em X.
    """
    print("="*88)
    print("ETAPA 1: Carregando e preparando os dados (VERSÃO FINAL)...")
    print("="*88)

    #! PATHS ----------------------------------------------------------------------
    # ATENÇÃO: REDIFINIR AQUI OS PATHS PARA A SUA MÁQUINA
    path_starcard = r'C:\Users\Leo\Desktop\Porto_Real\portoauto\src\originadores\data\StarCard.xlsx'
    path_originadores = r'C:\Users\Leo\Desktop\Porto_Real\portoauto\src\originadores\data\Originadores.xlsx'
    #------------------------------------------------------------------------------

    # ---------- Leitura ----------
    df_starcard = pd.read_excel(path_starcard)
    cols_originadores = ['CCB', 'Data de Inclusão', 'Prazo', 'Valor Parcela', 'Produto', 'Convênio', 'CAPAG']
    # CAPAG pode não existir; vamos ler com try simples
    try:
        df_originadores = pd.read_excel(path_originadores, usecols=cols_originadores)
    except ValueError:
        # <-- SUGESTÃO IMPLEMENTADA
        print("[ALERTA] Coluna 'CAPAG' não encontrada em Originadores.xlsx. Esta feature será desconsiderada no modelo.")
        cols_originadores = [c for c in cols_originadores if c != 'CAPAG']
        df_originadores = pd.read_excel(path_originadores, usecols=cols_originadores)

    # Normaliza datas e unicidade por CCB (primeira inclusão)
    if 'Data de Inclusão' in df_originadores.columns:
        df_originadores['Data de Inclusão'] = pd.to_datetime(df_originadores['Data de Inclusão'], errors='coerce')
        df_originadores = (
            df_originadores
            .sort_values(['CCB', 'Data de Inclusão'])
            .drop_duplicates(subset='CCB', keep='first')
        )

    # Merge
    df_merged = pd.merge(df_starcard, df_originadores, on='CCB', how='left', suffixes=('', '_orig'))

    # Renomeios
    df_merged = df_merged.rename(columns={
        'Data Referencia': 'DataGeracao',
        'Data Vencimento': 'DataVencimento',
        'ID Cliente': 'SacadoID'
    })

    # Tipos de data
    for col in ['DataGeracao', 'DataVencimento', 'Data de Nascimento']:
        if col in df_merged.columns:
            df_merged[col] = pd.to_datetime(df_merged[col], errors='coerce')

    # Conversão monetária
    if 'Valor Parcela' in df_merged.columns and df_merged['Valor Parcela'].dtype == 'object':
        s = df_merged['Valor Parcela'].astype(str)
        s = s.str.replace('R$', '', regex=False).str.replace('.', '', regex=False).str.replace(',', '.', regex=False).str.strip()
        df_merged['Valor Parcela'] = pd.to_numeric(s, errors='coerce')

    # Base
    df_base = df_merged.copy()
    del df_starcard, df_originadores, df_merged

    # Limpezas básicas
    if 'Produto' in df_base.columns:
        df_base['Produto'] = df_base['Produto'].fillna('')
    if 'Convênio' in df_base.columns:
        df_base['Convênio'] = df_base['Convênio'].fillna('')

    # Feature dias_de_atraso (com DataGeracao do snapshot; se DataGeracao é constante, vira “situação no dia da extração”)
    if 'DataGeracao' in df_base.columns and 'DataVencimento' in df_base.columns:
        df_base['dias_de_atraso'] = (df_base['DataGeracao'] - df_base['DataVencimento']).dt.days

    # Buckets de produto/empregado/convênio
    condicoes_produto = [
        df_base['Produto'].str.contains('Empréstimo', case=False, na=False),
        df_base['Produto'].str.contains('Cartão RMC', case=False, na=False),
        df_base['Produto'].str.contains('Cartão Benefício', case=False, na=False)
    ]
    opcoes_produto = ['Empréstimo', 'Cartão RMC', 'Cartão Benefício']
    df_base['_TipoProduto'] = np.select(condicoes_produto, opcoes_produto, default='Outros')

    condicoes_empregado = [
        df_base['Produto'].str.contains('Efetivo|Efetivio', case=False, na=False, regex=True),
        df_base['Produto'].str.contains('Temporário', case=False, na=False),
        df_base['Produto'].str.contains('CONTRATADO', case=False, na=False),
        df_base['Produto'].str.contains('Comissionado', case=False, na=False)
    ]
    opcoes_empregado = ['Efetivo', 'Temporário', 'Contratado', 'Comissionado']
    df_base['_TipoEmpregado'] = np.select(condicoes_empregado, opcoes_empregado, default='Outros')

    # Regex de convênio (ajustada para 'PREF' genérico; mantém 'AGN -' e 'GOV.')
    condicoes_convenio = [
        df_base['Convênio'].str.contains(r'GOV\.|AGN -', case=False, na=False, regex=True),
        df_base['Convênio'].str.contains(r'PREF', case=False, na=False, regex=True)
    ]
    opcoes_convenio = ['Estadual', 'Municipal']
    df_base['_EsferaConvenio'] = np.select(condicoes_convenio, opcoes_convenio, default='Outros')

    # ----- Relógio: Data de Inclusão -----
    if 'Data de Inclusão' not in df_base.columns:
        raise ValueError("Coluna 'Data de Inclusão' não encontrada após o merge. Verifique Originadores.xlsx.")

    # Ordenação temporal por inclusão para _MuitosContratos
    df_base = df_base.sort_values(['SacadoID', 'Data de Inclusão', 'CCB'])
    df_base['_NumContratosAnteriores'] = df_base.groupby('SacadoID').cumcount()
    df_base['_MuitosContratos'] = (df_base['_NumContratosAnteriores'] >= 2).astype(int)

    # Idade na Data de Inclusão (fallback em DataGeracao, se inclusão nula)
    if 'Data de Nascimento' in df_base.columns:
        base_idade = df_base['Data de Inclusão'].fillna(df_base.get('DataGeracao'))
        df_base['_IdadeCliente'] = (base_idade - df_base['Data de Nascimento']).dt.days / 365.25
        bins = [0, 37, 45, 53, 120]
        labels = ['Até 37 anos', '38 a 45 anos', '46 a 53 anos', '54 anos ou mais']
        df_base['_IdadesBins'] = pd.cut(df_base['_IdadeCliente'], bins=bins, labels=labels, right=True)

    # ---------- Labels por CCB ----------
    # Se tiver múltiplas linhas por CCB (parcelas/linhas), agregue o maior atraso
    if 'dias_de_atraso' not in df_base.columns:
        raise ValueError("'dias_de_atraso' não foi criado. Verifique 'DataGeracao' e 'DataVencimento'.")

    atraso_por_ccb = (
        df_base.groupby('CCB', as_index=False)['dias_de_atraso'].max()
        .rename(columns={'dias_de_atraso': 'max_dias_atraso'})
    )
    for k in [1, 30, 60, 90]:
        atraso_por_ccb[f'Vencido_{k}d_Flag'] = (atraso_por_ccb['max_dias_atraso'] >= k).astype(int)

    # ---------- Snapshot determinístico por CCB ----------
    # referência temporal: Data de Inclusão (fallback DataGeracao)
    df_base['DataInclRef'] = df_base['Data de Inclusão'].fillna(df_base.get('DataGeracao'))
    snapshots = df_base.sort_values(['CCB', 'DataInclRef', 'DataVencimento'])
    primeiro_snap = snapshots.groupby('CCB', as_index=False).first()

    features_para_modelo = [
        'CCB', 'Convênio', 'CAPAG', 'Prazo', 'Valor Parcela',
        '_TipoProduto', '_TipoEmpregado', '_EsferaConvenio', '_IdadesBins', '_MuitosContratos'
    ]
    ausentes = sorted(list(set(features_para_modelo) - set(primeiro_snap.columns)))
    if ausentes:
        print(f"[AVISO] Features ausentes e removidas (não encontradas no snapshot): {ausentes}")

    features_existentes = [f for f in features_para_modelo if f in primeiro_snap.columns]
    df_features = primeiro_snap[features_existentes]

    # df_ml final
    df_ml = pd.merge(df_features, atraso_por_ccb, on='CCB', how='inner').rename(columns={
        'Prazo': 'Total_Parcelas',
        'Valor Parcela': 'Valor_Parcela',
        '_EsferaConvenio': 'Esfera_Convenio',
        '_IdadesBins': 'Faixa_Idade',
        '_MuitosContratos': 'Muitos_Contratos',
        '_TipoProduto': 'Tipo_Produto',
        '_TipoEmpregado': 'Tipo_Empregado'
    })

    # Checagens úteis
    assert df_ml['CCB'].is_unique, "CCB não está único no df_ml! Revise o snapshot por CCB."
    assert df_ml[['Vencido_1d_Flag','Vencido_30d_Flag','Vencido_60d_Flag','Vencido_90d_Flag']].notna().all().all(), \
        "Algum label Vencido_* está com NaN."
    
    # <-- SUGESTÃO IMPLEMENTADA
    print(f"\nDataset para ML criado com {df_ml.shape[0]} linhas (CCBs) e {df_ml.shape[1]} colunas.")
    return df_ml


# =============================================================================
# ETAPA 2: TREINAMENTO E AVALIAÇÃO
# =============================================================================
def treinar_e_avaliar(df_ml):
    targets = ['Vencido_1d_Flag', 'Vencido_30d_Flag', 'Vencido_60d_Flag', 'Vencido_90d_Flag']
    resultados_finais = pd.DataFrame()

    for target in targets:
        print("\n" + "="*88)
        print(f"   TREINANDO MODELOS PARA O ALVO: {target}")
        print("="*88)

        # Define X e y removendo colunas proibidas
        X = df_ml.drop(columns=targets + ['CCB', 'max_dias_atraso'])
        y = df_ml[target]

        # Identifica colunas numéricas originais (antes de dummies)
        numerical_features = X.select_dtypes(include=np.number).columns.tolist()
        categorical_features = X.select_dtypes(include=['object', 'category']).columns.tolist()

        # Split estratificado
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.30, random_state=42, stratify=y
        )

        # Dummification APÓS o split + alinhamento
        X_train = pd.get_dummies(X_train, columns=categorical_features, drop_first=True)
        X_test  = pd.get_dummies(X_test,  columns=categorical_features, drop_first=True)
        X_test  = X_test.reindex(columns=X_train.columns, fill_value=0)

        # Guardas básicas
        print("#cols treino:", X_train.shape[1], "| #cols teste:", X_test.shape[1])
        assert list(X_train.columns) == list(X_test.columns), "Treino e teste desalinhados!"

        # Imputação + Escalonamento SOMENTE nas numéricas originais (interseção)
        num_cols = [c for c in numerical_features if c in X_train.columns]
        if len(num_cols) > 0:
            imputer = SimpleImputer(strategy='median')
            X_train[num_cols] = imputer.fit_transform(X_train[num_cols])
            X_test[num_cols]  = imputer.transform(X_test[num_cols])

            scaler = StandardScaler()
            X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
            X_test[num_cols]  = scaler.transform(X_test[num_cols])
        else:
            # <-- SUGESTÃO IMPLEMENTADA
            print("[INFO] Nenhuma coluna numérica foi identificada para imputação ou escalonamento.")

        # Log base rates
        print("Base rate no treino:", float(y_train.mean()).__round__(4), "| no teste:", float(y_test.mean()).__round__(4))

        # scale_pos_weight com guarda
        pos = int((y_train == 1).sum())
        neg = int((y_train == 0).sum())
        if pos == 0:
            print(f"[AVISO] Classe positiva ausente no treino para {target}. Ajustando scale_pos_weight=1.0.")
            scale_pos_weight = 1.0
        else:
            scale_pos_weight = neg / pos

        # Modelos
        models = {
            "Regressão Logística": LogisticRegression(random_state=42, class_weight='balanced', max_iter=1000),
            "Random Forest":       RandomForestClassifier(random_state=42, class_weight='balanced', n_jobs=-1),
            "SVM (Kernel RBF)":    SVC(kernel='rbf', probability=True, random_state=42, class_weight='balanced'),
            "XGBoost":             xgb.XGBClassifier(random_state=42, eval_metric='logloss',
                                                      use_label_encoder=False, scale_pos_weight=scale_pos_weight,
                                                      n_jobs=-1),
            "LightGBM":            lgb.LGBMClassifier(random_state=42, scale_pos_weight=scale_pos_weight,
                                                       n_jobs=-1)
        }

        resultados_target = {}
        for name, model in models.items():
            print(f"\n--- Treinando {name} ---")
            model.fit(X_train, y_train)
            # Probabilidades
            y_pred_proba = model.predict_proba(X_test)[:, 1]
            # Métricas
            auc = roc_auc_score(y_test, y_pred_proba)
            ap  = average_precision_score(y_test, y_pred_proba)
            resultados_target[name] = {'ROC-AUC': auc, 'PR-AUC(AP)': ap}
            print(f"ROC-AUC: {auc:.4f} | PR-AUC(AP): {ap:.4f}")

            # Importâncias — árvores/GBMs
            if isinstance(model, xgb.XGBClassifier):
                booster = model.get_booster()
                imp = booster.get_score(importance_type='gain')
                if len(imp) > 0:
                    importances = (pd.Series(imp).sort_values(ascending=False).head(10)
                                   .rename('Gain').reset_index().rename(columns={'index': 'Feature'}))
                    print("Top 10 importâncias (XGB, gain):")
                    print(importances)
            elif isinstance(model, lgb.LGBMClassifier):
                feats = model.feature_name_
                gains = model.booster_.feature_importance(importance_type='gain')
                importances = (pd.DataFrame({'Feature': feats, 'Gain': gains})
                               .sort_values('Gain', ascending=False).head(10))
                print("Top 10 importâncias (LGBM, gain):")
                print(importances)
            elif hasattr(model, 'feature_importances_'):
                importances = pd.DataFrame({
                    'Feature': X_train.columns,
                    'Importance': model.feature_importances_
                }).sort_values('Importance', ascending=False).head(10)
                print("Top 10 importâncias:")
                print(importances)
            elif isinstance(model, LogisticRegression):
                # <-- SUGESTÃO IMPLEMENTADA (Comentário)
                # A comparação da magnitude destes coeficientes é válida porque as features numéricas foram escalonadas.
                coefs = pd.Series(model.coef_[0], index=X_train.columns).sort_values(key=np.abs, ascending=False).head(10)
                print("Top 10 |coef| (Regressão Logística):")
                print(coefs)

        # Acumula resultados por alvo
        df_resultados_target = (pd.DataFrame(resultados_target).T)
        # guarda apenas ROC-AUC numa tabela comparativa “clássica”
        resultados_finais = pd.concat([resultados_finais, df_resultados_target['ROC-AUC'].rename(target)], axis=1)

        # Checagens adicionais úteis
        suspeitas = {'dias_de_atraso', 'max_dias_atraso', 'DataGeracao', 'DataVencimento', 'Data de Inclusão'}
        suspeitas_presentes = sorted(list(set(X_train.columns) & suspeitas))
        if suspeitas_presentes:
            print(f"[ALERTA] Colunas suspeitas presentes em X: {suspeitas_presentes}")

    return resultados_finais


# =============================================================================
# ETAPA 3: EXECUÇÃO
# =============================================================================
if __name__ == "__main__":
    df_ml = carregar_e_preparar_dados_corrigido()

    # Relatório rápido de base rates por label
    print("\nBase rates dos alvos (df_ml):")
    print(df_ml[['Vencido_1d_Flag','Vencido_30d_Flag','Vencido_60d_Flag','Vencido_90d_Flag']].mean().round(4))

    resultados_finais = treinar_e_avaliar(df_ml)

    print("\n" + "="*88)
    print("   RESUMO COMPARATIVO FINAL - ROC-AUC")
    print("="*88)
    print(resultados_finais.round(4))

    melhor_modelo_geral = resultados_finais.mean(axis=1).idxmax()
    print("\n" + "-"*88)
    print(f"Considerando a média de ROC-AUC em todos os alvos, o melhor modelo parece ser: '{melhor_modelo_geral}'")
    print("-"*88)

ETAPA 1: Carregando e preparando os dados (VERSÃO FINAL)...

Dataset para ML criado com 6838 linhas (CCBs) e 15 colunas.

Base rates dos alvos (df_ml):
Vencido_1d_Flag     0.8157
Vencido_30d_Flag    0.6897
Vencido_60d_Flag    0.5294
Vencido_90d_Flag    0.3353
dtype: float64

   TREINANDO MODELOS PARA O ALVO: Vencido_1d_Flag
#cols treino: 43 | #cols teste: 43
Base rate no treino: 0.8157 | no teste: 0.8158

--- Treinando Regressão Logística ---
ROC-AUC: 0.9372 | PR-AUC(AP): 0.9819
Top 10 |coef| (Regressão Logística):
Tipo_Empregado_Contratado         -4.054948
Convênio_PREF. ANANINDEUA         -3.682524
Tipo_Empregado_Temporário          3.488191
Tipo_Empregado_Efetivo             2.488744
Convênio_PREF. CAMPOS DO JORDÃO    2.305191
Tipo_Produto_Empréstimo           -1.713887
Convênio_PREF. COTIA               1.706626
Convênio_PREF. EMBU DAS ARTES     -1.546657
Convênio_PREF. SANTA LUZIA         1.468440
Esfera_Convenio_Municipal         -1.463444
dtype: float64

--- Treinando Random Fo

In [23]:
# -*- coding: utf-8 -*-
"""
Script de Treinamento e Avaliação de Modelos de Machine Learning
VERSÃO CORRIGIDA - COM VALIDAÇÃO TEMPORAL E SEM DATA LEAKAGE
"""
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import seaborn as sns

# Modelos e Ferramentas
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import lightgbm as lgb

# Métricas
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc

warnings.filterwarnings('ignore')
sns.set_theme(style="whitegrid")

# =============================================================================
# ETAPA 1: CARREGAMENTO E PREPARAÇÃO DOS DADOS
# ALTERAÇÃO: Removida a criação da feature '_MuitosContratos' para evitar data leakage.
# ALTERAÇÃO: Garante que o dataframe final esteja ordenado por data.
# =============================================================================
def carregar_e_preparar_dados():
    """
    Carrega, limpa e prepara os dados, usando a "Data de Inclusão" para
    estabelecer a ordem cronológica correta dos contratos.
    """
    print("="*80)
    print("ETAPA 1: Carregando e preparando os dados...")
    print("="*80)

    # ! PATHS ----------------------------------------------------------------------
    path_starcard = r'C:\Users\Leo\Desktop\Porto_Real\portoauto\src\originadores\data\StarCard.xlsx'
    path_originadores = r'C:\Users\Leo\Desktop\Porto_Real\portoauto\src\originadores\data\Originadores.xlsx'
    # ------------------------------------------------------------------------------

    df_starcard = pd.read_excel(path_starcard)
    cols_originadores = ['CCB', 'Prazo', 'Valor Parcela', 'Produto', 'Convênio', 'Data de Inclusão']
    df_originadores = pd.read_excel(path_originadores, usecols=cols_originadores)

    if not df_originadores['CCB'].is_unique:
        df_originadores = df_originadores.drop_duplicates(subset='CCB', keep='first')

    df_merged = pd.merge(df_starcard, df_originadores, on='CCB', how='left', suffixes=('', '_orig'))
    df_merged = df_merged.rename(columns={
        'Data Referencia': 'DataGeracao', 'Data Vencimento': 'DataVencimento',
        'ID Cliente': 'SacadoID'
    })

    cols_monetarias = ['Valor Parcela']
    for col in cols_monetarias:
        if df_merged[col].dtype == 'object':
            df_merged[col] = df_merged[col].astype(str).str.replace('R$', '', regex=False).str.replace('.', '', regex=False).str.replace(',', '.', regex=False).str.strip()
            df_merged[col] = pd.to_numeric(df_merged[col], errors='coerce')

    cols_data = ['DataGeracao', 'DataVencimento', 'Data de Nascimento', 'Data de Inclusão']
    for col in cols_data:
        df_merged[col] = pd.to_datetime(df_merged[col], errors='coerce')

    df_base = df_merged.copy()
    del df_starcard, df_originadores, df_merged

    df_base['dias_de_atraso'] = (df_base['DataGeracao'] - df_base['DataVencimento']).dt.days
    df_base['Produto'] = df_base['Produto'].fillna('')
    df_base['Convênio'] = df_base['Convênio'].fillna('')
    
    condicoes_produto = [df_base['Produto'].str.contains('Empréstimo', case=False, na=False), df_base['Produto'].str.contains('Cartão RMC', case=False, na=False), df_base['Produto'].str.contains('Cartão Benefício', case=False, na=False)]
    opcoes_produto = ['Empréstimo', 'Cartão RMC', 'Cartão Benefício']
    df_base['_TipoProduto'] = np.select(condicoes_produto, opcoes_produto, default='Outros')

    condicoes_empregado = [df_base['Produto'].str.contains('Efetivo|Efetivio', case=False, na=False, regex=True), df_base['Produto'].str.contains('Temporário', case=False, na=False), df_base['Produto'].str.contains('CONTRATADO', case=False, na=False), df_base['Produto'].str.contains('Comissionado', case=False, na=False)]
    opcoes_empregado = ['Efetivo', 'Temporário', 'Contratado', 'Comissionado']
    df_base['_TipoEmpregado'] = np.select(condicoes_empregado, opcoes_empregado, default='Outros')

    condicoes_convenio = [df_base['Convênio'].str.contains(r'GOV\.|AGN -', case=False, na=False, regex=True), df_base['Convênio'].str.contains(r'PREF\.|PRERF', case=False, na=False, regex=True)]
    opcoes_convenio = ['Estadual', 'Municipal']
    df_base['_EsferaConvenio'] = np.select(condicoes_convenio, opcoes_convenio, default='Outros')

    if 'Data de Nascimento' in df_base.columns:
        df_base['_IdadeCliente'] = ((df_base['DataGeracao'] - df_base['Data de Nascimento']).dt.days / 365.25)
        bins = [0, 37, 45, 53, 120]
        labels = ['Até 37 anos', '38 a 45 anos', '46 a 53 anos', '54 anos ou mais']
        df_base['_IdadesBins'] = pd.cut(df_base['_IdadeCliente'], bins=bins, labels=labels, right=True)
    
    # REMOVIDO DAQUI: A feature '_MuitosContratos' será criada dentro do loop de validação cruzada.

    atraso_por_ccb = df_base.groupby('CCB')['dias_de_atraso'].max().reset_index()
    atraso_por_ccb.rename(columns={'dias_de_atraso': 'max_dias_atraso'}, inplace=True)
    atraso_por_ccb['Vencido_1d_Flag'] = (atraso_por_ccb['max_dias_atraso'] >= 1).astype(int)
    atraso_por_ccb['Vencido_30d_Flag'] = (atraso_por_ccb['max_dias_atraso'] >= 30).astype(int)
    atraso_por_ccb['Vencido_60d_Flag'] = (atraso_por_ccb['max_dias_atraso'] >= 60).astype(int)
    atraso_por_ccb['Vencido_90d_Flag'] = (atraso_por_ccb['max_dias_atraso'] >= 90).astype(int)

    # A feature 'SacadoID' é mantida por enquanto para ser usada na criação da feature '_MuitosContratos'
    features_para_modelo = ['CCB', 'SacadoID', 'Convênio', 'CAPAG', 'Prazo', 'Valor Parcela', '_TipoProduto', '_TipoEmpregado', '_EsferaConvenio', '_IdadesBins', 'Data de Inclusão']
    features_existentes = [f for f in features_para_modelo if f in df_base.columns]
    df_features = df_base[features_existentes].drop_duplicates(subset='CCB', keep='first')

    df_ml = pd.merge(df_features, atraso_por_ccb, on='CCB')
    
    df_ml = df_ml.rename(columns={
        'Prazo': 'Total_Parcelas', 'Valor Parcela': 'Valor_Parcela', '_EsferaConvenio': 'Esfera_Convenio',
        '_IdadesBins': 'Faixa_Idade', '_TipoProduto': 'Tipo_Produto', '_TipoEmpregado': 'Tipo_Empregado',
        'Data de Inclusão': 'DataInclusao'
    })

    # IMPORTANTE: Ordenar os dados por data para a validação temporal funcionar corretamente
    df_ml = df_ml.sort_values('DataInclusao').reset_index(drop=True)
    
    print(f"Dataset para ML criado com {len(df_ml)} contratos (CCBs).")
    print(f"Período dos dados: de {df_ml['DataInclusao'].min().date()} a {df_ml['DataInclusao'].max().date()}")
    return df_ml


# =============================================================================
# ETAPA 2: FUNÇÕES DE DIAGNÓSTICO E TREINAMENTO
# =============================================================================

def treinar_e_avaliar_modelos_temporal_cv(df_ml, targets, n_splits=5):
    """
    Executa um pipeline de treinamento usando Validação Cruzada Temporal (TimeSeriesSplit)
    para garantir uma avaliação robusta e sem vazamento de dados do futuro.
    A feature '_MuitosContratos' é criada dentro de cada fold para evitar data leakage.
    """
    resultados_finais = {}

    for target in targets:
        print("\n" + "="*80)
        print(f" TREINANDO MODELOS PARA O ALVO: {target}")
        print("="*80)
        
        y = df_ml[target]
        
        if y.nunique() < 2:
            print(f"AVISO: O dataset inteiro para o alvo '{target}' tem apenas uma classe. Pulando.")
            continue
        
        # Prepara o X inicial, mantendo colunas necessárias para feature engineering
        cols_to_drop_initial = targets + ['max_dias_atraso', 'DataInclusao']
        X = df_ml.drop(columns=[c for c in cols_to_drop_initial if c in df_ml.columns])

        # Define os modelos a serem treinados
        # Os hiperparâmetros foram mantidos da versão original, podem ser otimizados
        models = {
            "Regressão Logística": Pipeline([('scaler', StandardScaler()), ('model', LogisticRegression(random_state=42, class_weight='balanced', C=0.1, solver='liblinear'))]),
            "Random Forest": RandomForestClassifier(random_state=42, class_weight='balanced', n_jobs=-1, max_depth=5, n_estimators=150, min_samples_leaf=10),
            "XGBoost": xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss', n_jobs=-1, max_depth=4, learning_rate=0.05, n_estimators=200, subsample=0.8, colsample_bytree=0.8),
            "LightGBM": lgb.LGBMClassifier(random_state=42, n_jobs=-1, max_depth=4, learning_rate=0.05, n_estimators=200, subsample=0.8, colsample_bytree=0.8)
        }
        
        resultados_target = {}

        for name, model in models.items():
            print(f"\n--- Modelo: {name} ---")
            
            # Listas para armazenar as métricas de cada fold
            roc_auc_scores = []
            auprc_inadimplentes_scores = []
            auprc_adimplentes_scores = []

            # Configura a validação cruzada temporal
            tscv = TimeSeriesSplit(n_splits=n_splits)

            for fold, (train_index, test_index) in enumerate(tscv.split(X)):
                print(f"  - Processando Fold {fold+1}/{n_splits}...")
                
                # Divisão de treino e teste para este fold
                X_train, X_test = X.iloc[train_index].copy(), X.iloc[test_index].copy()
                y_train, y_test = y.iloc[train_index].copy(), y.iloc[test_index].copy()

                # --- CORREÇÃO DE DATA LEAKAGE NA FEATURE ENGINEERING ---
                # 1. Aprender a regra SOMENTE com os dados de treino
                sacado_contratos_treino = X_train.groupby('SacadoID')['CCB'].nunique()
                sacado_alto_treino = sacado_contratos_treino[sacado_contratos_treino >= 3].index
                # 2. Aplicar a regra aprendida no treino e no teste
                X_train['Muitos_Contratos'] = X_train['SacadoID'].isin(sacado_alto_treino).astype(int)
                X_test['Muitos_Contratos'] = X_test['SacadoID'].isin(sacado_alto_treino).astype(int)

                # Remover colunas de ID que não são features
                X_train = X_train.drop(columns=['CCB', 'SacadoID'])
                X_test = X_test.drop(columns=['CCB', 'SacadoID'])

                # One-hot encoding e alinhamento de colunas
                categorical_features = X_train.select_dtypes(include=['object', 'category']).columns
                X_train = pd.get_dummies(X_train, columns=categorical_features, drop_first=True)
                X_test = pd.get_dummies(X_test, columns=categorical_features, drop_first=True)
                X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)

                # Tratamento de desbalanceamento para XGBoost/LightGBM
                if 'scale_pos_weight' in model.get_params() if not isinstance(model, Pipeline) else 'model__scale_pos_weight' in model.get_params():
                    try:
                        scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()
                        model.set_params(**{'model__scale_pos_weight' if isinstance(model, Pipeline) else 'scale_pos_weight': scale_pos_weight})
                    except ZeroDivisionError:
                        print("    AVISO: Nenhuma amostra da classe positiva no treino. Pulando fold.")
                        continue
                
                # Treinamento do modelo
                model.fit(X_train, y_train)
                y_pred_proba = model.predict_proba(X_test)[:, 1]

                # Cálculo das métricas para o fold
                roc_auc_scores.append(roc_auc_score(y_test, y_pred_proba))
                
                # AUPRC para inadimplentes (classe positiva, geralmente o foco)
                precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
                auprc_inadimplentes_scores.append(auc(recall, precision))

                # AUPRC para adimplentes (classe negativa)
                precision, recall, _ = precision_recall_curve(y_test, 1 - y_pred_proba, pos_label=0)
                auprc_adimplentes_scores.append(auc(recall, precision))

            # Média e desvio padrão das métricas após todos os folds
            resultados_target[name] = {
                'ROC-AUC Média': np.mean(roc_auc_scores),
                'ROC-AUC Desv.': np.std(roc_auc_scores),
                'AUPRC_inadimplentes': np.mean(auprc_inadimplentes_scores),
                'AUPRC_adimplentes': np.mean(auprc_adimplentes_scores)
            }
            print(f"  - Resultado Final (Média CV): ROC-AUC={np.mean(roc_auc_scores):.4f}, AUPRC_inadimplentes={np.mean(auprc_inadimplentes_scores):.4f}")

        resultados_finais[target] = pd.DataFrame.from_dict(resultados_target, orient='index')
        
    return resultados_finais


# =============================================================================
# ETAPA 3: EXECUÇÃO PRINCIPAL E EXIBIÇÃO DO RESUMO FINAL
# =============================================================================
if __name__ == "__main__":
    df_ml = carregar_e_preparar_dados()
    targets_a_prever = ['Vencido_1d_Flag', 'Vencido_30d_Flag', 'Vencido_60d_Flag', 'Vencido_90d_Flag']
    
    # Executa a nova função de treinamento com validação temporal
    resultados_finais = treinar_e_avaliar_modelos_temporal_cv(df_ml, targets_a_prever, n_splits=5)
    
    print("\n" + "="*80)
    print("  RESUMO COMPARATIVO FINAL (VALIDAÇÃO TEMPORAL CRUZADA)")
    print("="*80)

    if not resultados_finais:
        print("Nenhum alvo pôde ser avaliado.")
    else:
        for target, df_resultado in resultados_finais.items():
            print(f"\n--- Resultados para o Alvo: {target} ---")
            print(df_resultado.round(4))
            
            if not df_resultado.empty:
                # Sugestão: Selecionar o melhor modelo com base no AUPRC da classe de interesse (inadimplentes)
                melhor_modelo = df_resultado['AUPRC_inadimplentes'].idxmax()
                melhor_score = df_resultado['AUPRC_inadimplentes'].max()
                
                print("\n" + "-"*80)
                print(f"Considerando a performance na identificação de INADIMPLENTES (maior AUPRC),")
                print(f"o melhor modelo parece ser: '{melhor_modelo}' com AUPRC médio de {melhor_score:.4f}")
                print("-"*80)

ETAPA 1: Carregando e preparando os dados...
Dataset para ML criado com 6838 contratos (CCBs).
Período dos dados: de 2024-09-18 a 2025-08-05

 TREINANDO MODELOS PARA O ALVO: Vencido_1d_Flag

--- Modelo: Regressão Logística ---
  - Processando Fold 1/5...


ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: np.int64(1)

In [22]:
# Após treinar o seu modelo XGBoost final para um alvo
# Supondo que 'model' seja o seu objeto XGBoost treinado
importances = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': model.feature_importances_
}).sort_values('Importance', ascending=False)

print("Top 10 Features Mais Importantes:")
print(importances.head(10))

ValueError: All arrays must be of the same length

In [9]:
# -*- coding: utf-8 -*-
"""
Script Completo e Metodologicamente Sólido para Modelagem de Risco de Crédito

Este script implementa um fluxo de trabalho de Machine Learning robusto, incorporando
as seguintes melhores práticas:

1.  **Snapshot t₀:** As features de cada contrato são extraídas de sua data de origem (t₀),
    garantindo que o modelo não tenha acesso a informações futuras.

2.  **Pipeline de Pré-processamento:** Utiliza o Pipeline e o ColumnTransformer do Scikit-learn para
    encapsular todas as etapas de transformação de dados (imputação, encoding, escalonamento).
    Isso previne o vazamento de dados (data leakage) e torna o modelo pronto para produção.

3.  **Validação Temporal:** A divisão dos dados em treino e teste é feita com base no tempo
    (holdout temporal). O modelo é treinado com contratos mais antigos e avaliado em
    contratos mais recentes, simulando um cenário real de previsão.

4.  **Avaliação Completa:** Calcula tanto a ROC-AUC quanto a PR-AUC (Precision-Recall),
    que é especialmente útil para classes desbalanceadas.

5.  **Modularidade:** Itera sobre múltiplos horizontes de inadimplência (1d, 30d, 60d, 90d)
    e treina um portfólio de modelos de classificação para cada um.
"""
import pandas as pd
import numpy as np
import os

# Modelos e Ferramentas de Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb
import lightgbm as lgb

# Métricas de Avaliação
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc

# =============================================================================
# ETAPA 1: CARREGAMENTO E PREPARAÇÃO DOS DADOS COM SNAPSHOT t₀
# =============================================================================
def carregar_dados_com_snapshot_t0():
    """
    Carrega, limpa e transforma os dados, garantindo que as features de cada contrato
    sejam extraídas de um snapshot na sua data de origem (t₀).
    """
    print("="*80)
    print("ETAPA 1: Carregando e preparando os dados com Snapshot em t₀...")
    print("="*80)

    #! PATHS ----------------------------------------------------------------------
    # ATENÇÃO: REDIFINIR AQUI OS PATHS PARA A SUA MÁQUINA
    path_starcard = r'C:\Users\Leo\Desktop\Porto_Real\portoauto\src\originadores\data\StarCard.xlsx'
    path_originadores = r'C:\Users\Leo\Desktop\Porto_Real\portoauto\src\originadores\data\Originadores.xlsx'
    #------------------------------------------------------------------------------

    # Carregamento e limpeza inicial
    df_starcard = pd.read_excel(path_starcard)
    cols_originadores = ['CCB', 'Prazo', 'Valor Parcela', 'Produto', 'Convênio']
    df_originadores = pd.read_excel(path_originadores, usecols=cols_originadores)
    if not df_originadores['CCB'].is_unique:
        df_originadores = df_originadores.drop_duplicates(subset='CCB', keep='first')
    df_merged = pd.merge(df_starcard, df_originadores, on='CCB', how='left', suffixes=('', '_orig'))
    df_merged = df_merged.rename(columns={'Data Referencia': 'DataGeracao', 'Data Vencimento': 'DataVencimento', 'ID Cliente': 'SacadoID', 'Data Aquisicao': 'DataAquisicao'})
    
    cols_monetarias = ['Valor Parcela']
    for col in cols_monetarias:
        if df_merged[col].dtype == 'object':
            df_merged[col] = df_merged[col].astype(str).str.replace('R$', '', regex=False).str.replace('.', '', regex=False).str.replace(',', '.', regex=False).str.strip()
            df_merged[col] = pd.to_numeric(df_merged[col], errors='coerce')
            
    cols_data = ['DataGeracao', 'DataVencimento', 'Data de Nascimento', 'DataAquisicao']
    for col in cols_data:
        df_merged[col] = pd.to_datetime(df_merged[col], errors='coerce')
    df_base = df_merged.copy()

    # 1. Definir t₀ (data de origem) para cada CCB
    df_base['t0'] = df_base.groupby('CCB')['DataAquisicao'].transform('min')
    
    # 2. Capturar o estado final do contrato para criar o alvo
    df_base['dias_de_atraso'] = (df_base['DataGeracao'] - df_base['DataVencimento']).dt.days
    estado_final = df_base.groupby('CCB')['dias_de_atraso'].max().reset_index()
    estado_final = estado_final.rename(columns={'dias_de_atraso': 'max_dias_atraso'})
    
    # 3. Filtrar para manter apenas a linha de features de t₀
    df_snapshot = df_base.loc[df_base.groupby('CCB')['DataAquisicao'].idxmin()].copy()

    # 4. Engenharia de Features usando o Snapshot de t₀
    df_snapshot['Produto'] = df_snapshot['Produto'].fillna('')
    df_snapshot['Convênio'] = df_snapshot['Convênio'].fillna('')
    condicoes_produto = [df_snapshot['Produto'].str.contains('Empréstimo', case=False, na=False), df_snapshot['Produto'].str.contains('Cartão RMC', case=False, na=False), df_snapshot['Produto'].str.contains('Cartão Benefício', case=False, na=False)]
    opcoes_produto = ['Empréstimo', 'Cartão RMC', 'Cartão Benefício']
    df_snapshot['_TipoProduto'] = np.select(condicoes_produto, opcoes_produto, default='Outros')
    condicoes_empregado = [df_snapshot['Produto'].str.contains('Efetivo|Efetivio', case=False, na=False, regex=True), df_snapshot['Produto'].str.contains('Temporário', case=False, na=False), df_snapshot['Produto'].str.contains('CONTRATADO', case=False, na=False), df_snapshot['Produto'].str.contains('Comissionado', case=False, na=False)]
    opcoes_empregado = ['Efetivo', 'Temporário', 'Contratado', 'Comissionado']
    df_snapshot['_TipoEmpregado'] = np.select(condicoes_empregado, opcoes_empregado, default='Outros')
    condicoes_convenio = [df_snapshot['Convênio'].str.contains(r'GOV\.|AGN -', case=False, na=False, regex=True), df_snapshot['Convênio'].str.contains(r'PREF\.|PRERF', case=False, na=False, regex=True)]
    opcoes_convenio = ['Estadual', 'Municipal']
    df_snapshot['_EsferaConvenio'] = np.select(condicoes_convenio, opcoes_convenio, default='Outros')
    
    if 'Data de Nascimento' in df_snapshot.columns:
        df_snapshot['_IdadeCliente'] = ((df_snapshot['t0'] - df_snapshot['Data de Nascimento']).dt.days / 365.25)
        bins = [0, 37, 45, 53, 120]
        labels = ['Até 37 anos', '38 a 45 anos', '46 a 53 anos', '54 anos ou mais']
        df_snapshot['_IdadesBins'] = pd.cut(df_snapshot['_IdadeCliente'], bins=bins, labels=labels, right=True)

    sacado_contratos = df_base.groupby('SacadoID')['CCB'].nunique()
    sacado_contratos_alto = sacado_contratos[sacado_contratos >= 3].index
    df_snapshot['_MuitosContratos'] = df_snapshot['SacadoID'].isin(sacado_contratos_alto).astype(int)

    # 5. Juntar as features de t₀ com os alvos do estado final
    df_ml = pd.merge(df_snapshot, estado_final, on='CCB', how='left')

    # Criar todos os alvos
    df_ml['Vencido_1d_Flag'] = (df_ml['max_dias_atraso'] >= 1).astype(int)
    df_ml['Vencido_30d_Flag'] = (df_ml['max_dias_atraso'] >= 30).astype(int)
    df_ml['Vencido_60d_Flag'] = (df_ml['max_dias_atraso'] >= 60).astype(int)
    df_ml['Vencido_90d_Flag'] = (df_ml['max_dias_atraso'] >= 90).astype(int)

    df_ml = df_ml.rename(columns={
        'Prazo': 'Total_Parcelas', 'Valor Parcela': 'Valor_Parcela', '_EsferaConvenio': 'Esfera_Convenio',
        '_IdadesBins': 'Faixa_Idade', '_MuitosContratos': 'Muitos_Contratos',
        '_TipoProduto': 'Tipo_Produto', '_TipoEmpregado': 'Tipo_Empregado'
    })
    
    print("Dataset para ML criado com snapshot t₀ correto.")
    return df_ml

# =============================================================================
# Bloco Principal de Execução
# =============================================================================
if __name__ == "__main__":
    df_ml = carregar_dados_com_snapshot_t0()

    # --- Configurações da Modelagem ---
    targets = ['Vencido_1d_Flag', 'Vencido_30d_Flag', 'Vencido_60d_Flag', 'Vencido_90d_Flag']
    features_to_use = [
        'Total_Parcelas', 'Valor_Parcela', 'Muitos_Contratos', 'CAPAG',
        'Convênio', 'Tipo_Produto', 'Tipo_Empregado', 'Esfera_Convenio', 'Faixa_Idade'
    ]
    
    # Garantir que apenas colunas existentes sejam usadas
    features_to_use = [f for f in features_to_use if f in df_ml.columns]
    
    resultados_finais = pd.DataFrame()

    # --- Divisão Temporal (Temporal Holdout) ---
    df_ml = df_ml.sort_values('t0').dropna(subset=['t0'])
    cutoff_date = df_ml['t0'].quantile(0.7, interpolation='nearest')
    train_df = df_ml[df_ml['t0'] <= cutoff_date]
    test_df = df_ml[df_ml['t0'] > cutoff_date]

    print("\n" + "="*80)
    print("ETAPA 2: Divisão Temporal dos Dados")
    print("="*80)
    print(f"Divisão temporal realizada na data: {cutoff_date.date()}")
    print(f"Dados de treino: {len(train_df)} amostras (de {train_df['t0'].min().date()} até {train_df['t0'].max().date()})")
    print(f"Dados de teste: {len(test_df)} amostras (de {test_df['t0'].min().date()} até {test_df['t0'].max().date()})")

    # --- Construção do Pipeline de Pré-processamento ---
    numerical_features = train_df[features_to_use].select_dtypes(include=np.number).columns.tolist()
    categorical_features = train_df[features_to_use].select_dtypes(include=['object', 'category']).columns.tolist()

    numerical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())])

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, numerical_features),
            ('cat', categorical_transformer, categorical_features)],
        remainder='passthrough')

    # --- Loop de Treinamento e Avaliação ---
    for target in targets:
        print("\n" + "="*80)
        print(f"   TREINANDO MODELOS PARA O ALVO: {target} (Validação Temporal)")
        print("="*80)

        X_train = train_df[features_to_use]
        y_train = train_df[target]
        X_test = test_df[features_to_use]
        y_test = test_df[target]
        
        # Define os modelos
        models = {
            "Regressão Logística": LogisticRegression(random_state=42, class_weight='balanced', max_iter=1000, n_jobs=-1),
            "Random Forest": RandomForestClassifier(random_state=42, class_weight='balanced', n_jobs=-1),
            "XGBoost": xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss', n_jobs=-1),
            "LightGBM": lgb.LGBMClassifier(random_state=42, class_weight='balanced', n_jobs=-1)
        }

        target_results = {}
        for name, model in models.items():
            print(f"\n--- Treinando {name} ---")
            
            pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                       ('classifier', model)])
            
            pipeline.fit(X_train, y_train)
            
            y_pred_proba = pipeline.predict_proba(X_test)[:, 1]
            
            roc_auc = roc_auc_score(y_test, y_pred_proba)
            precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
            pr_auc = auc(recall, precision)
            
            target_results[f"{name} (ROC-AUC)"] = roc_auc
            target_results[f"{name} (PR-AUC)"] = pr_auc
            
            print(f"ROC-AUC Score: {roc_auc:.4f}")
            print(f"PR-AUC Score: {pr_auc:.4f}")

            # Extração e exibição da importância das features
            if name in ["Random Forest", "XGBoost", "LightGBM"]:
                try:
                    # Nomes das features após o one-hot encoding
                    ohe_feature_names = pipeline.named_steps['preprocessor'].named_transformers_['cat'].named_steps['onehot'].get_feature_names_out(categorical_features)
                    final_feature_names = numerical_features + list(ohe_feature_names)
                    
                    importances_values = pipeline.named_steps['classifier'].feature_importances_
                    
                    importances_df = pd.DataFrame({
                        'Feature': final_feature_names,
                        'Importance': importances_values
                    }).sort_values('Importance', ascending=False).head(10)
                    
                    print("Top 10 Features mais importantes:")
                    print(importances_df)
                except Exception as e:
                    print(f"Não foi possível extrair a importância das features: {e}")

        # Armazenar resultados
        df_target_results = pd.DataFrame.from_dict(target_results, orient='index', columns=[target])
        if resultados_finais.empty:
            resultados_finais = df_target_results
        else:
            resultados_finais = resultados_finais.join(df_target_results, how='outer')

    # =============================================================================
    # ETAPA FINAL: EXIBIÇÃO DO RESUMO COMPARATIVO
    # =============================================================================
    print("\n" + "="*80)
    print("   RESUMO COMPARATIVO FINAL - MÉTRICAS DE AVALIAÇÃO (Validação Temporal)")
    print("="*80)
    print(resultados_finais.round(4))

ETAPA 1: Carregando e preparando os dados com Snapshot em t₀...
Dataset para ML criado com snapshot t₀ correto.

ETAPA 2: Divisão Temporal dos Dados
Divisão temporal realizada na data: 2025-06-06
Dados de treino: 4807 amostras (de 2024-09-23 até 2025-06-06)
Dados de teste: 2031 amostras (de 2025-06-09 até 2025-08-05)

   TREINANDO MODELOS PARA O ALVO: Vencido_1d_Flag (Validação Temporal)

--- Treinando Regressão Logística ---
ROC-AUC Score: 0.6588
PR-AUC Score: 0.5191

--- Treinando Random Forest ---
ROC-AUC Score: 0.5792
PR-AUC Score: 0.7110
Top 10 Features mais importantes:
                             Feature  Importance
18  Convênio_PREF. JUAZEIRO DO NORTE    0.281647
0                     Total_Parcelas    0.117335
40          Esfera_Convenio_Estadual    0.089289
41         Esfera_Convenio_Municipal    0.073208
8                Convênio_GOV. GOIAS    0.069239
1                      Valor_Parcela    0.061993
5                            CAPAG_C    0.039859
39         Tipo_Empregado